# * TF 경고 끄기 *
---

In [1]:
import logging
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger('tensorflow').setLevel(logging.FATAL)

In [2]:
import tensorflow as tf

# 데이터셋
---

### 원본 데이터셋 목록 생성
---

In [3]:
import os
import pandas as pd

def read_raw_dataset_list(path):
    train_csv = pd.read_csv(os.path.join(path, 'train.csv'))
    test_csv = pd.read_csv(os.path.join(path, 'test.csv'))
    
    train_input_all_files = os.path.join(path, 'train_input_img') + os.path.sep + train_csv['input_img']
    train_label_all_files = os.path.join(path, 'train_label_img') + os.path.sep + train_csv['label_img']
    
    test_all_files = os.path.join(path, 'test_input_img') + os.path.sep + test_csv['input_img']
    submission_all_files = os.path.join(path, 'test_input_img') + os.path.sep + test_csv['submission_name']
    
    train_input_files = train_input_all_files[60:].to_numpy()
    train_label_files = train_label_all_files[60:].to_numpy()
    val_input_files = train_input_all_files[:60].to_numpy()
    val_label_files = train_label_all_files[:60].to_numpy()
    return train_input_files, train_label_files, val_input_files, val_label_files, test_all_files, submission_all_files

### 데이터셋 생성기
---

In [4]:
import os
import numpy as np

from glob import glob

import pandas as pd
import tensorflow as tf
from sklearn.utils import shuffle

def train_map_func(inp_path, targ_path):
    inp = np.load(inp_path)
    inp = np.squeeze(inp)
    inp = np.stack((inp, ) * 3, axis=-1)
    inp = inp.astype(np.float32) / 255
    targ = np.load(targ_path)
    targ = np.squeeze(targ)
    targ = np.stack((targ, ) * 3, axis=-1)
    targ = targ.astype(np.float32) / 255
    inp, targ = augmentation(inp, targ)
    return inp, targ


def val_map_func(inp_path, targ_path):
    inp = np.load(inp_path)
    inp = np.squeeze(inp)
    inp = np.stack((inp, ) * 3, axis=-1)
    inp = inp.astype(np.float32) / 255
    targ = np.load(targ_path)
    targ = np.squeeze(targ)
    targ = np.stack((targ, ) * 3, axis=-1)
    targ = targ.astype(np.float32) / 255
    return inp, targ


def augmentation(inp, targ):
    inp, targ = random_rot(inp, targ)
    inp, targ = random_flip(inp, targ)
    return inp, targ


def random_rot(inp, targ):
    k = np.random.randint(4)
    inp = np.rot90(inp, k)
    targ = np.rot90(targ, k)
    return inp, targ


def random_flip(inp, targ):
    f = np.random.randint(2)
    if f == 0:
        inp = np.fliplr(inp)
        targ = np.fliplr(targ)
    return inp, targ


def create_dataset_generator(path, batch_size, image_size, stride):
    # 전처리된 npy 포맷 데이터(shape=[image_size, image_size, 1]) 불러오기
    train_input_files = glob(os.path.join(path, f'train_input_stride_{stride}_patch_{image_size}_intensity', '*.npy'))
    train_label_files = glob(os.path.join(path, f'train_label_stride_{stride}_patch_{image_size}_intensity', '*.npy'))
    val_input_files = glob(os.path.join(path, f'val_input_stride_{stride}_patch_{image_size}_intensity', '*.npy'))
    val_label_files = glob(os.path.join(path, f'val_labels_stride_{stride}_patch_{image_size}_intensity', '*.npy'))
    train_input_files, train_label_files = shuffle(train_input_files, train_label_files, random_state=42)

    train_dataset = tf.data.Dataset.from_tensor_slices((train_input_files, train_label_files))
    train_dataset = train_dataset.map(lambda i1, i2: tf.numpy_function(train_map_func, [i1, i2], [tf.float32, tf.float32]),
                                      num_parallel_calls=tf.data.experimental.AUTOTUNE)
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices((val_input_files, val_label_files))
    val_dataset = val_dataset.map(lambda i1, i2: tf.numpy_function(val_map_func, [i1, i2], [tf.float32, tf.float32]),
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)
    val_dataset = val_dataset.batch(batch_size)
    val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return train_dataset, val_dataset

### 데이터셋 가공
---
원본 데이터셋을 stride만큼 image_size x image_size 정방형 영상으로 나눕니다.

#### 데이터셋 가공 함수

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

def make_image_as_intensity_patches(image_path_list, save_name, image_size, stride):
    save_path = f'{save_name}_stride_{stride}_patch_{image_size}_intensity'
    os.makedirs(save_path, exist_ok=True)
    
    num = 0
    for path in tqdm(image_path_list):
        # 이미지 로딩 후, HSI(HSV)로 변환 그리고 I채널만 추출
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        hsv_channels = cv2.split(hsv_img)
        intensity_img = hsv_channels[2]
        for top in range(0, intensity_img.shape[0], stride):
            for left in range(0, intensity_img.shape[1], stride):
                patch = intensity_img[top:top + image_size, left:left + image_size]
                piece = np.zeros([image_size, image_size, 1], np.uint8)
                piece[:patch.shape[0], :patch.shape[1]] = patch[:, :, np.newaxis]
                np.save(os.path.join(save_path, f'{num}.npy'), piece.copy())
                num += 1

#### 원본 데이터셋 파일 목록 불러오기

In [ ]:
import os

train_input_files, train_label_files, val_input_files, val_label_files, test_all_files, _ = read_raw_dataset_list(os.path.join('.', 'LG_Raw_Data'))

In [ ]:
print(train_input_files)
print(train_label_files)

#### 데이터셋 가공 매개변수

In [ ]:
image_size = 256
stride = 128

#### 데이터셋 가공 수행

In [ ]:
import os

print('Processing train inputs...')
make_image_as_intensity_patches(train_input_files, os.path.join('.', 'LG_Intensity_Only_Data', 'train_input'), image_size, stride)
print('Processing train labels...')
make_image_as_intensity_patches(train_label_files, os.path.join('.', 'LG_Intensity_Only_Data', 'train_label'), image_size, stride)
print('Processing validation inputs...')
make_image_as_intensity_patches(val_input_files, os.path.join('.', 'LG_Intensity_Only_Data', 'val_input'), image_size, stride)
print('Processing validation labels...')
make_image_as_intensity_patches(val_label_files, os.path.join('.', 'LG_Intensity_Only_Data', 'val_labels'), image_size, stride)

# 모델 학습
---

## 모델 정의
---

In [5]:
def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = tf.keras.layers.Conv2D(filters, size, strides=strides, padding=padding)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if activation == True:
        x = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
    return x


def residual_block(blockInput, num_filters=16):
    x = tf.keras.layers.LeakyReLU(alpha=0.1)(blockInput)
    x = tf.keras.layers.BatchNormalization()(x)
    blockInput = tf.keras.layers.BatchNormalization()(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = tf.keras.layers.Add()([x, blockInput])
    return x


def ResUNet101V2(image_size, weights='imagenet', dropout_rate=0.1, start_neurons=16):
    input_shape=(image_size, image_size, 3)
    backbone = tf.keras.applications.ResNet101V2(weights=weights, include_top=False, input_shape=input_shape)
    input_layer = backbone.input

    conv4 = backbone.layers[122].output
    conv4 = tf.keras.layers.LeakyReLU(alpha=0.1)(conv4)
    pool4 = tf.keras.layers.MaxPooling2D((2, 2))(conv4)
    pool4 = tf.keras.layers.Dropout(dropout_rate)(pool4)

    convm = tf.keras.layers.Conv2D(start_neurons * 32, (3, 3), activation=None, padding='same')(pool4)
    convm = residual_block(convm, start_neurons * 32)
    convm = residual_block(convm, start_neurons * 32)
    convm = tf.keras.layers.LeakyReLU(alpha=0.1)(convm)

    deconv4 = tf.keras.layers.Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding='same')(convm)
    uconv4 = tf.keras.layers.concatenate([deconv4, conv4])
    uconv4 = tf.keras.layers.Dropout(dropout_rate)(uconv4)

    uconv4 = tf.keras.layers.Conv2D(start_neurons * 16, (3, 3), activation=None, padding='same')(uconv4)
    uconv4 = residual_block(uconv4, start_neurons * 16)
    uconv4 = residual_block(uconv4, start_neurons * 16)
    uconv4 = tf.keras.layers.LeakyReLU(alpha=0.1)(uconv4)

    deconv3 = tf.keras.layers.Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding='same')(uconv4)
    conv3 = backbone.layers[76].output
    uconv3 = tf.keras.layers.concatenate([deconv3, conv3])
    uconv3 = tf.keras.layers.Dropout(dropout_rate)(uconv3)

    uconv3 = tf.keras.layers.Conv2D(start_neurons * 8, (3, 3), activation=None, padding='same')(uconv3)
    uconv3 = residual_block(uconv3, start_neurons * 8)
    uconv3 = residual_block(uconv3, start_neurons * 8)
    uconv3 = tf.keras.layers.LeakyReLU(alpha=0.1)(uconv3)

    deconv2 = tf.keras.layers.Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding='same')(uconv3)
    conv2 = backbone.layers[30].output
    uconv2 = tf.keras.layers.concatenate([deconv2, conv2])

    uconv2 = tf.keras.layers.Dropout(0.1)(uconv2)
    uconv2 = tf.keras.layers.Conv2D(start_neurons * 4, (3, 3), activation=None, padding='same')(uconv2)
    uconv2 = residual_block(uconv2, start_neurons * 4)
    uconv2 = residual_block(uconv2, start_neurons * 4)
    uconv2 = tf.keras.layers.LeakyReLU(alpha=0.1)(uconv2)

    deconv1 = tf.keras.layers.Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding='same')(uconv2)
    conv1 = backbone.layers[2].output
    uconv1 = tf.keras.layers.concatenate([deconv1, conv1])

    uconv1 = tf.keras.layers.Dropout(0.1)(uconv1)
    uconv1 = tf.keras.layers.Conv2D(start_neurons * 2, (3, 3), activation=None, padding='same')(uconv1)
    uconv1 = residual_block(uconv1, start_neurons * 2)
    uconv1 = residual_block(uconv1, start_neurons * 2)
    uconv1 = tf.keras.layers.LeakyReLU(alpha=0.1)(uconv1)

    uconv0 = tf.keras.layers.Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding='same')(uconv1)
    uconv0 = tf.keras.layers.Dropout(0.1)(uconv0)
    uconv0 = tf.keras.layers.Conv2D(start_neurons * 1, (3, 3), activation=None, padding='same')(uconv0)
    uconv0 = residual_block(uconv0, start_neurons * 1)
    uconv0 = residual_block(uconv0, start_neurons * 1)
    uconv0 = tf.keras.layers.LeakyReLU(alpha=0.1)(uconv0)

    uconv0 = tf.keras.layers.Dropout(dropout_rate / 2)(uconv0)
    output_layer = tf.keras.layers.Conv2D(3, (1, 1), padding='same', activation='sigmoid')(uconv0)

    model = tf.keras.models.Model(input_layer, output_layer)

    return model

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import concatenate
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, LeakyReLU, Dropout, MaxPooling2D, Conv2DTranspose


def conv2d(filters: int):
    return Conv2D(filters=filters, kernel_size=(3, 3),
                  padding='same', kernel_regularizer=l2(0.), bias_regularizer=l2(0.))


def conv2d_transpose(filters: int):
    return Conv2DTranspose(filters=filters, kernel_size=(2, 2), strides=(2, 2), padding='same')


def create_unetpp_model(image_size, channels=1, num_filters=2, dropout_rate=0, alpha=0.01):
    model_input = Input((image_size, image_size, channels))
    x00 = conv2d(filters=int(16 * num_filters))(model_input)
    x00 = BatchNormalization()(x00)
    x00 = LeakyReLU(alpha)(x00)
    x00 = Dropout(dropout_rate)(x00)
    x00 = conv2d(filters=int(16 * num_filters))(x00)
    x00 = BatchNormalization()(x00)
    x00 = LeakyReLU(alpha)(x00)
    x00 = Dropout(dropout_rate)(x00)
    p0 = MaxPooling2D(pool_size=(2, 2))(x00)

    x10 = conv2d(filters=int(32 * num_filters))(p0)
    x10 = BatchNormalization()(x10)
    x10 = LeakyReLU(alpha)(x10)
    x10 = Dropout(dropout_rate)(x10)
    x10 = conv2d(filters=int(32 * num_filters))(x10)
    x10 = BatchNormalization()(x10)
    x10 = LeakyReLU(alpha)(x10)
    x10 = Dropout(dropout_rate)(x10)
    p1 = MaxPooling2D(pool_size=(2, 2))(x10)

    x01 = conv2d_transpose(int(16 * num_filters))(x10)
    x01 = concatenate([x00, x01])
    x01 = conv2d(filters=int(16 * num_filters))(x01)
    x01 = BatchNormalization()(x01)
    x01 = LeakyReLU(alpha)(x01)
    x01 = conv2d(filters=int(16 * num_filters))(x01)
    x01 = BatchNormalization()(x01)
    x01 = LeakyReLU(alpha)(x01)
    x01 = Dropout(dropout_rate)(x01)

    x20 = conv2d(filters=int(64 * num_filters))(p1)
    x20 = BatchNormalization()(x20)
    x20 = LeakyReLU(alpha)(x20)
    x20 = Dropout(dropout_rate)(x20)
    x20 = conv2d(filters=int(64 * num_filters))(x20)
    x20 = BatchNormalization()(x20)
    x20 = LeakyReLU(alpha)(x20)
    x20 = Dropout(dropout_rate)(x20)
    p2 = MaxPooling2D(pool_size=(2, 2))(x20)

    x11 = conv2d_transpose(int(16 * num_filters))(x20)
    x11 = concatenate([x10, x11])
    x11 = conv2d(filters=int(16 * num_filters))(x11)
    x11 = BatchNormalization()(x11)
    x11 = LeakyReLU(alpha)(x11)
    x11 = conv2d(filters=int(16 * num_filters))(x11)
    x11 = BatchNormalization()(x11)
    x11 = LeakyReLU(alpha)(x11)
    x11 = Dropout(dropout_rate)(x11)

    x02 = conv2d_transpose(int(16 * num_filters))(x11)
    x02 = concatenate([x00, x01, x02])
    x02 = conv2d(filters=int(16 * num_filters))(x02)
    x02 = BatchNormalization()(x02)
    x02 = LeakyReLU(alpha)(x02)
    x02 = conv2d(filters=int(16 * num_filters))(x02)
    x02 = BatchNormalization()(x02)
    x02 = LeakyReLU(alpha)(x02)
    x02 = Dropout(dropout_rate)(x02)

    x30 = conv2d(filters=int(128 * num_filters))(p2)
    x30 = BatchNormalization()(x30)
    x30 = LeakyReLU(alpha)(x30)
    x30 = Dropout(dropout_rate)(x30)
    x30 = conv2d(filters=int(128 * num_filters))(x30)
    x30 = BatchNormalization()(x30)
    x30 = LeakyReLU(alpha)(x30)
    x30 = Dropout(dropout_rate)(x30)
    p3 = MaxPooling2D(pool_size=(2, 2))(x30)

    x21 = conv2d_transpose(int(16 * num_filters))(x30)
    x21 = concatenate([x20, x21])
    x21 = conv2d(filters=int(16 * num_filters))(x21)
    x21 = BatchNormalization()(x21)
    x21 = LeakyReLU(alpha)(x21)
    x21 = conv2d(filters=int(16 * num_filters))(x21)
    x21 = BatchNormalization()(x21)
    x21 = LeakyReLU(alpha)(x21)
    x21 = Dropout(dropout_rate)(x21)

    x12 = conv2d_transpose(int(16 * num_filters))(x21)
    x12 = concatenate([x10, x11, x12])
    x12 = conv2d(filters=int(16 * num_filters))(x12)
    x12 = BatchNormalization()(x12)
    x12 = LeakyReLU(alpha)(x12)
    x12 = conv2d(filters=int(16 * num_filters))(x12)
    x12 = BatchNormalization()(x12)
    x12 = LeakyReLU(alpha)(x12)
    x12 = Dropout(dropout_rate)(x12)

    x03 = conv2d_transpose(int(16 * num_filters))(x12)
    x03 = concatenate([x00, x01, x02, x03])
    x03 = conv2d(filters=int(16 * num_filters))(x03)
    x03 = BatchNormalization()(x03)
    x03 = LeakyReLU(alpha)(x03)
    x03 = conv2d(filters=int(16 * num_filters))(x03)
    x03 = BatchNormalization()(x03)
    x03 = LeakyReLU(alpha)(x03)
    x03 = Dropout(dropout_rate)(x03)

    m = conv2d(filters=int(256 * num_filters))(p3)
    m = BatchNormalization()(m)
    m = LeakyReLU(alpha)(m)
    m = conv2d(filters=int(256 * num_filters))(m)
    m = BatchNormalization()(m)
    m = LeakyReLU(alpha)(m)
    m = Dropout(dropout_rate)(m)

    x31 = conv2d_transpose(int(128 * num_filters))(m)
    x31 = concatenate([x31, x30])
    x31 = conv2d(filters=int(128 * num_filters))(x31)
    x31 = BatchNormalization()(x31)
    x31 = LeakyReLU(alpha)(x31)
    x31 = conv2d(filters=int(128 * num_filters))(x31)
    x31 = BatchNormalization()(x31)
    x31 = LeakyReLU(alpha)(x31)
    x31 = Dropout(dropout_rate)(x31)

    x22 = conv2d_transpose(int(64 * num_filters))(x31)
    x22 = concatenate([x22, x20, x21])
    x22 = conv2d(filters=int(64 * num_filters))(x22)
    x22 = BatchNormalization()(x22)
    x22 = LeakyReLU(alpha)(x22)
    x22 = conv2d(filters=int(64 * num_filters))(x22)
    x22 = BatchNormalization()(x22)
    x22 = LeakyReLU(alpha)(x22)
    x22 = Dropout(dropout_rate)(x22)

    x13 = conv2d_transpose(int(32 * num_filters))(x22)
    x13 = concatenate([x13, x10, x11, x12])
    x13 = conv2d(filters=int(32 * num_filters))(x13)
    x13 = BatchNormalization()(x13)
    x13 = LeakyReLU(alpha)(x13)
    x13 = conv2d(filters=int(32 * num_filters))(x13)
    x13 = BatchNormalization()(x13)
    x13 = LeakyReLU(alpha)(x13)
    x13 = Dropout(dropout_rate)(x13)

    x04 = conv2d_transpose(int(16 * num_filters))(x13)
    x04 = concatenate([x04, x00, x01, x02, x03], axis=3)
    x04 = conv2d(filters=int(16 * num_filters))(x04)
    x04 = BatchNormalization()(x04)
    x04 = LeakyReLU(alpha)(x04)
    x04 = conv2d(filters=int(16 * num_filters))(x04)
    x04 = BatchNormalization()(x04)
    x04 = LeakyReLU(alpha)(x04)
    x04 = Dropout(dropout_rate)(x04)
    output = tf.keras.layers.Conv2D(channels, (1, 1), padding='same', activation='sigmoid')(x04)   # For reconstruction
    return tf.keras.Model(inputs=[model_input], outputs=[output])

## 학습 매개변수 선언
---

In [11]:
image_size = 256
stride = 128
epoch = 1
batch_size = 8

psnr_weight = 1
ssim_weight = 1
recon_weight = 1

learning_rate = 1e-5

model_filename = f'ResUNet101V2_psnr_ssim_recon_input_{image_size}_epoch_{epoch}_best.h5'
model_filename_last = f'ResUNet101V2_psnr_ssim_recon_input_{image_size}_epoch_{epoch}_last.h5'

## 학습 데이터셋 생성기 생성
---

In [7]:
train_generator, validate_generator = create_dataset_generator('.\\LG_Intensity_Only_Data', batch_size, image_size, stride)

## 모델 생성 (새로 학습)
---

#### 사용자 정의 로스 함수

In [12]:
def ssim_loss(y_true, y_pred):
    return 1. - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))

def psnr_loss(y_true, y_pred):
    return tf.keras.losses.mean_squared_error(y_true, y_pred)

def recon_loss(y_true, y_pred):
    return tf.keras.losses.mean_absolute_error(y_true, y_pred)

def create_psnr_ssim_recon_combined_loss(psnr_weight, ssim_weight, recon_weight):
    def psnr_ssim_recon_combined_loss(y_true, y_pred):
        return (psnr_weight * psnr_loss(y_true, y_pred)) + (ssim_weight * ssim_loss(y_true, y_pred)) + (recon_weight * recon_loss(y_true, y_pred))
    return psnr_ssim_recon_combined_loss

combined_loss = create_psnr_ssim_recon_combined_loss(psnr_weight, ssim_weight, recon_weight)

In [13]:
# model = create_unetpp_model(image_size, num_filters=num_filters, dropout_rate=dropout_rate, alpha=alpha)
model = ResUNet101V2(image_size)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss=combined_loss)

## 모델 생성 (재학습)
---

In [ ]:
model = tf.keras.models.load_model(model_filename, custom_objects={combined_loss.__name__: combined_loss})

## 모델 학습 및 저장
---

In [ ]:
model.fit(train_generator, epochs=epoch, validation_data=validate_generator, callbacks=[
    tf.keras.callbacks.ModelCheckpoint(
        filepath=model_filename,
        monitor='val_loss',
        save_best_only=True,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=model_filename_last,
        save_freq=512
    )
])

 3385/19468 [====>.........................] - ETA: 52:59 - loss: 0.7811

# 모델 추론
---

## 학습된 모델 불러오기
---

In [ ]:
model = tf.keras.models.load_model(model_filename, custom_objects={combined_loss.__name__: combined_loss})

## 모델 추론 함수 선언
---

In [ ]:
import cv2
import numpy as np
from tqdm import tqdm


def predict(img_paths, image_size, stride=32, batch_size=128):
    results = []
    for img_path in img_paths:
        ori_img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        hsv_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2HSV)
        hsv_channels = cv2.split(hsv_img)
        intensity_img = hsv_channels[2]
        intensity_img = np.stack((intensity_img, ) * 3, axis=-1)
        
        img = intensity_img.astype(np.float32) / 255
        crop = []
        position = []
        batch_count = 0
        
        result_img = np.zeros_like(img)
        voting_mask = np.zeros_like(img)
        for top in tqdm(range(0, img.shape[0], stride)):
            for left in range(0, img.shape[1], stride):
                piece = np.zeros([image_size, image_size, 3], np.float32)
                temp = img[top:top + image_size, left:left + image_size, :]
                piece[:temp.shape[0], :temp.shape[1], :] = temp
                crop.append(piece)
                position.append([top, left])
                batch_count += 1
                if batch_count == batch_size:
                    crop = np.array(crop)
                    pred = model(crop) * 255
                    crop = []
                    batch_count=0
                    for num, (t, l) in enumerate(position):
                        piece = pred[num]
                        h, w, c = result_img[t:t + image_size, l:l + image_size,:].shape
                        result_img[t:t + image_size, l:l + image_size,:] += piece[:h, :w]
                        voting_mask[t:t + image_size, l:l + image_size, :] += 1
                    position = []
                    
        result_img = result_img / voting_mask
        result_img = result_img.astype(np.uint8)
        
        hsv_channels[2] = np.squeeze(cv2.cvtColor(result_img, cv2.COLOR_BGR2GRAY))
        merged_hsv_img = cv2.merge(hsv_channels)
        merged_img = cv2.cvtColor(merged_hsv_img, cv2.COLOR_HSV2BGR)
        results.append((result_img, merged_img))
        
    return results

## 영상 품질 평가 함수 선언
---

In [ ]:
import math
import numpy as np


def rmse_score(true, pred):
    score = math.sqrt(np.mean((true - pred) ** 2))
    return score

def psnr_score(true, pred, pixel_max):
    score = 20 * np.log10(pixel_max / rmse_score(true,pred))
    return score

## 추론 매개변수 선언
---

In [ ]:
image_size = 256
stride = 128
batch_size = 16

## 검증/테스트 데이터셋 목록 불러오기
---

In [ ]:
train_input_files, train_label_files, val_input_files, val_label_files, test_all_files, _ = read_raw_dataset_list(os.path.join('.', 'LG_Raw_Data'))

## 추론 및 결과보기 (Validate)
---

In [ ]:
import cv2
my_slice = 1, 2

input_images = [cv2.imread(fn, cv2.IMREAD_COLOR) for fn in val_input_files[my_slice[0]:my_slice[1]]]
label_images = [cv2.imread(fn, cv2.IMREAD_COLOR) for fn in val_label_files[my_slice[0]:my_slice[1]]]
predicted_images = predict(val_input_files[my_slice[0]:my_slice[1]], image_size, stride, batch_size)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.subplot(1, 4, 1)
plt.imshow(input_images[0])
plt.title('Input', fontsize=10)
plt.subplot(1, 4, 2)
plt.imshow(label_images[0])
plt.title('Label', fontsize=10)
plt.subplot(1, 4, 3)
plt.imshow(predicted_images[0][0], cmap='gray')
plt.title('Predicted (Intensity Only)', fontsize=10)
plt.subplot(1, 4, 4)
plt.imshow(predicted_images[0][1])
plt.title('Predicted', fontsize=10)

print(f'PSNR: {psnr_score(label_images[0], predicted_images[0][1], 255)}')